In [112]:
import gurobipy as gurobi
import pandas as pd
import numpy as np

In [ ]:
df_request = pd.read_csv('../data/02_input_target.csv')
products = df_request["Product"].unique()
countries = df_request["Country"].unique()

# COSTI DI PRODUZIONE

df_prod_costs = pd.read_csv('../data/03_input_productionCost.csv')

products_plan = [] # key for the dictionary
products_costs = {}

for index, row in df_prod_costs.iterrows():
    s = row["Country"]
    p = row["Product"]
    c = row["Unit Cost"]
    products_costs[f"{s}-{p}"] = c
    products_plan.append(f"{s}-{p}")

# COSTI DI TRANSPORTO

df_transport_costs = pd.read_csv('../data/02_03_input_shipmentsCost.csv')

transports_plan = [] # key for the dictionary
transports_cost = {}

for index, row in df_transport_costs.iterrows():
    o = row["Origin"]
    d = row["Destination"]
    c = row["Unit Cost"]  
    transports_cost[f"{o}-{d}"] = c
    for p in products: 
        transports_plan.append(f"{o}-{d}-{p}")

# CAPACITA'

df_capacity = pd.read_csv('../data/02_input_capacity.csv')
capacities = {}

for index, row in df_capacity.iterrows():
    s = row["Country"]
    c = row["Monthly Capacity"]
    capacities[s] = c

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
prod_sol = {f"{c}-{p}-{m}2004" : 0 for c in countries for p in products for m in months}
transport_sol = {f"{o}-{d}-{p}-{m}2004" : 0 for o in countries for d in countries for p in products for m in months}
for m in months:

    # MONTHLY REQUEST
    df_month_request = df_request.loc[df_request["Month"] == f"{m}2004"]
    requests = {}
    for index, row in df_month_request.iterrows():
        c = row["Country"]
        p = row["Product"]
        r = row["Quantity"]
        requests[f"{c}-{p}"] = r

    model = gurobi.Model(f"{m}")
    # PIANO DI PRODUZIONE
    g_production = {}
    
    for key in products_plan:
        g_production[key] = model.addVar(name = key, lb=0, vtype = gurobi.GRB.INTEGER)

    # PIANO DI TRASPORTO
    g_transport = {}
    
    for key in transports_plan:
        g_transport[key] = model.addVar(name = key, lb = 0, vtype = gurobi.GRB.INTEGER)
    
    # VINCOLI DI CAPACITA'
    capacity_constrs = {}
    for c in countries:
        capacity_constrs[f"country : {c} - CAPACITY CONSTRAINT"] = model.addConstr(gurobi.quicksum(g_production[f"{c}-{p}"] for p in products) <= capacities[c], name = f"country : {c} - CAPACITY CONSTRAINT") # capacity constraint 
    print(capacities)

    # VINCOLI DI DOMANDA
    requests_constrs = {}  
    for p in products:
        for c in countries:
            
            # EXPORT COUNTRIES
            export_countries = []
            for d in countries:
                if f"{c}-{d}-{p}" in transports_plan:
                    export_countries.append(f"{c}-{d}-{p}")
            # IMPORT COUNTRIES
            import_countries = []
            for d in countries:
                if f"{d}-{c}-{p}" in transports_plan:
                    import_countries.append(f"{d}-{c}-{p}")
            
            proof = 0 if c != "France" else 100
            constr = model.addConstr((g_production[f"{c}-{p}"] - gurobi.quicksum(g_transport[key] for key in export_countries) + gurobi.quicksum(g_transport[key] for key in import_countries)) >= (requests[f"{c}-{p}"] - proof), name = f"country : {c} - product : {p} - REQUEST CONSTRAINT")
            requests_constrs[f"country : {c} - product : {p} - REQUEST CONSTRAINT"] = constr
    
    # VINCOLI DI FLUSSO
    flux_constrs = {}
    for p in products:
        for c in countries:
            
            # EXPORT COUNTRIES
            export_countries = []
            for d in countries:
                if f"{c}-{d}-{p}" in transports_plan:
                    export_countries.append(f"{c}-{d}-{p}")
            # IMPORT COUNTRIES
            import_countries = []
            for d in countries:
                if f"{d}-{c}-{p}" in transports_plan:
                    import_countries.append(f"{d}-{c}-{p}")
            
            constr = model.addConstr((g_production[f"{c}-{p}"] - gurobi.quicksum(g_transport[key] for key in export_countries) + gurobi.quicksum(g_transport[key] for key in import_countries)) >= 0, name = " {c} FLUX CONSTRAINT")

    production_objective = gurobi.quicksum(products_costs[key]*g_production[key] for key in products_plan)
    transport_objective = gurobi.quicksum(transports_cost["-".join(key.split("-")[:2])]*g_transport[key] for key in transports_plan)
    model.setObjective(production_objective + transport_objective, gurobi.GRB.MINIMIZE) 
    model.optimize()
    
    print(f"{m}")
    """
    model.computeIIS()
    
    for ci in model.getConstrs():
        if ci.getAttr("IISConstr") == 1:
            print(f"{ci.ConstrName} : {ci.getAttr("IISConstr")}")
    """
    if model.status == gurobi.GRB.OPTIMAL:
        print(f"Soluzione ottima trovata:")
        for key, var in g_production.items():
            new_key = key+f"-{m}2004"
            prod_sol[new_key] = var.X
        for key, var in g_transport.items():
            new_key = key+f"-{m}2004"
            transport_sol[new_key] = var.X
        print(f"Valore ottimo: {model.ObjVal}")
    else:
        print("Nessuna soluzione ottima trovata.")
    print()
output = open("lp_model_product_plan.csv")
for key in prod_sol:
    output.write(f"{key}-{prod_sol[key]}\n")
output.close()
output = open("lp_model_transport_plan.csv")  
for key in transport_sol:
    output.write(f"{key}-{transport_sol[key]}\n")
output.close()

{'Australia': 105000, 'Brazil': 90000, 'Egypt': 90000, 'France': 30000, 'India': 90000, 'Italy': 85000, 'Japan': 45000000, 'Russia': 105000, 'South Africa': 75000, 'United Kingdom': 55000}
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2010 rows, 5500 columns and 21000 nonzeros
Model fingerprint: 0xad7190d5
Variable types: 0 continuous, 5500 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 5e+07]
Found heuristic solution: objective 9584256.2700
Presolve removed 1052 rows and 549 columns
Presolve time: 0.05s
Presolved: 958 rows, 4951 columns, 9843 nonzeros
Variable types: 0 continuous, 4951 integer (0 binary)
Found heuristic solution: o

TypeError: 'set' object does not support item assignment